In [1]:
import time
import pandas as pd
from datetime import date
from datetime import datetime
from datetime import timedelta
import win32com.client as win32
import os
start_time = time.time()

## Parte 1: Ingreso nombre archivos

In [2]:
OnHandAvance = '2023-09-15 ON HAND AVANCE.xlsb'
OnHandCompleto_DiaAnterior = '2023-09-14 ON HAND COMPLETO.xlsb'
afo = '2023-09-15 AFO AVANCE.xlsb'
plan_crc = 'Plan Septiembre 2023 CRC.xlsx'
plan_crecsar = 'Plan CSAR-CRE_ORIGINAL.xlsx'
activacion = 'ACTIVACION 15-09.XLSX'
qpluss_ecc = 'QPLUSS ECC 15-09.XLSX'
qpluss = 'QPLUS (1).xlsx'

## Parte 2: Carga archivos

In [3]:
# 1.1 Leer Data On Hand Avance día Actual
df1 = pd.read_excel(OnHandAvance, engine = 'pyxlsb', dtype = 'string', usecols = 'A:X')

# 1.2 Al cargar la data, python lee las comas como puntos, por lo tanto aplicamos la funcion replace para mantenerlas como ','
df1['Ctd.'] = df1['Ctd.'].str.replace(".",",", regex = False)

# 1.3 Agregar OS+OP+MAT
df1['OS+OP+MAT'] = df1['OS'] + df1['OP'] + df1['Producto']

# 1.4 Agregamos identificador unico
df1['Identificador Unico'] = df1.index.values.tolist()

## Parte 3: Agregar Columnas

## Días en Proceso

In [4]:
# 2.1 Fecha a formato fecha, creando una columna auxiliar para aquellos valores sin Tipo fecha
df1['Fecha EM_Aux'] = df1['Fecha EM']
df1['Fecha EM'] = pd.to_datetime(df1['Fecha EM'], errors='coerce', dayfirst = True)

# 2.2 Calculo días en proceso
today = datetime.today()
df1['DIAS EN PROCESO'] = list(map(lambda x: ((today - x).days), df1['Fecha EM']))

# 2.3 Fecha EM a String
df1['Fecha EM'] = df1['Fecha EM'].dt.strftime("%d/%m/%Y")

# 2.4 Aquellos que no tenian formato fecha , los igualamos a la columna Auxiliar para recuperar dichos valores
df1.loc[pd.isna(df1['Fecha EM']) == True, 'Fecha EM'] = df1['Fecha EM_Aux'].iloc[df1.loc[pd.isna(df1['Fecha EM'])].index.values]

# 2.5 Borrar Fecha Auxiliar
df1.drop(labels = 'Fecha EM_Aux', axis = 1, inplace = True)

## Plan

In [5]:
# 3.1 Insertar Columna Plan
df1['Plan'] = pd.NA

# 3.2 Crear Columnas auxiliares 4 primeros digitos (CRC, CSAR, CRE) y 6 primeros digitos (KIT)
df1['OS.SUP_AUX'] = list(map(lambda x: x[:4], df1['OS.SUP']))
df1['OS.SUP_KIT'] = list(map(lambda x: x[:6], df1['OS.SUP']))

# 3.3 Replacements
df1.loc[df1['OS.SUP_AUX'] == '5028', 'Plan'] = 'PLAN CSAR'
df1.loc[df1['OS.SUP_AUX'] == '5082', 'Plan'] = 'PLAN CRE'
df1.loc[df1['OS.SUP_AUX'] == '5059', 'Plan'] = 'PLAN CRC'
df1.loc[df1['OS.SUP_KIT'] == '000001', 'Plan'] = 'PLAN KIT'

# 3.4 Eliminar Columnas Auxiliares
df1.drop(labels = ['OS.SUP_AUX', 'OS.SUP_KIT'], axis = 1, inplace = True)

## Fecha de Despacho

In [6]:
# 4. Leer archivos
df_crc = pd.read_excel(plan_crc, sheet_name = 'Planificación CRC', usecols = ['Orden de Servicio', 'Fecha de Entrega'],
                       dtype = 'string')
df_cre = pd.read_excel(plan_crecsar, sheet_name = 'Planificación CRE', usecols = ['Orden de Servicio','Fecha de Entrega'], 
                      dtype = 'string' )
df_csar = pd.read_excel(plan_crecsar, sheet_name = 'Planificación CSAR', usecols = ['Oso','Fecha de Entrega'],
                       dtype = 'string')

In [7]:
# CRC

# 5.1 Fechas CRC: Mas reciente al mas Antiguo
df_crc.sort_values(by = 'Fecha de Entrega', ascending = False, inplace = True)

# 5.2 Reset index
df_crc.reset_index(inplace = True)

# 5.3 Columna auxiliar para arreglar formato CRC
df_crc['Fecha de Entrega_Auxiliar'] = df_crc['Fecha de Entrega']

# 5.4 Fecha Entrega a formato fecha (String dtype)
df_crc['Fecha de Entrega'] = pd.to_datetime(df_crc['Fecha de Entrega'], errors = 'coerce').dt.strftime('%d/%m/%Y')

# 5.5 Aquellos que no tenian formato fecha , los igualamos a la columna Auxiliar para recuperar dichos valores
df_crc.loc[pd.isna(df_crc['Fecha de Entrega']) == True, 'Fecha de Entrega'] = \
df_crc['Fecha de Entrega_Auxiliar'].iloc[df_crc.loc[pd.isna(df_crc['Fecha de Entrega'])].index.values]

# 5.6 Eliminar columna Auxiliar e index
df_crc.drop(['Fecha de Entrega_Auxiliar', 'index'], axis = 1, inplace = True)

# 5.7 Eliminar duplicados Orden de Servicio
df_crc.drop_duplicates('Orden de Servicio', keep = 'first', inplace = True)

In [8]:
# CRE

# 6.1 Fechas CRC: Mas reciente al mas Antiguo
df_cre.sort_values(by = 'Fecha de Entrega', ascending = False, inplace = True)

# 6.2 Reset index
df_cre.reset_index(inplace = True)

# 6.3 Columna auxiliar para arreglar formato CRC
df_cre['Fecha de Entrega_Auxiliar'] = df_cre['Fecha de Entrega']

# 6.4 Fecha Entrega a formato fecha (String dtype)
df_cre['Fecha de Entrega'] = pd.to_datetime(df_cre['Fecha de Entrega'], errors = 'coerce').dt.strftime('%d/%m/%Y')

# 6.5 Aquellos que no tenian formato fecha , los igualamos a la columna Auxiliar para recuperar dichos valores
df_cre.loc[pd.isna(df_cre['Fecha de Entrega']) == True, 'Fecha de Entrega'] = \
df_cre['Fecha de Entrega_Auxiliar'].iloc[df_cre.loc[pd.isna(df_cre['Fecha de Entrega'])].index.values]

# 6.6 Eliminar columna Auxiliar e index
df_cre.drop(['Fecha de Entrega_Auxiliar', 'index'], axis = 1, inplace = True)

# 6.7 Eliminar duplicados Orden de Servicio
df_cre.drop_duplicates('Orden de Servicio', keep = 'first', inplace = True)

In [9]:
# CSAR

# 7.1 Fechas CRC: Mas reciente al mas Antiguo
df_csar.sort_values(by = 'Fecha de Entrega', ascending = False, inplace = True)

# 7.2 Reset index
df_csar.reset_index(inplace = True)

# 7.3 Columna auxiliar para arreglar formato CRC
df_csar['Fecha de Entrega_Auxiliar'] = df_csar['Fecha de Entrega']

# 7.4 Fecha Entrega a formato fecha (String dtype)
df_csar['Fecha de Entrega'] = pd.to_datetime(df_csar['Fecha de Entrega'], errors = 'coerce').dt.strftime('%d/%m/%Y')

# 7.5 Aquellos que no tenian formato fecha , los igualamos a la columna Auxiliar para recuperar dichos valores
df_csar.loc[pd.isna(df_csar['Fecha de Entrega']) == True, 'Fecha de Entrega'] = \
df_csar['Fecha de Entrega_Auxiliar'].iloc[df_csar.loc[pd.isna(df_csar['Fecha de Entrega'])].index.values]

# 7.6 Eliminar columna Auxiliar e index
df_csar.drop(['Fecha de Entrega_Auxiliar', 'index'], axis = 1, inplace = True)

# 7.7 Eliminar duplicados Orden de Servicio
df_csar.drop_duplicates('Oso', keep = 'first', inplace = True)

# 7.8 Eliminar N/A
df_csar.dropna(axis = 0, subset = 'Oso', inplace = True)

# 7.9 Renombrar columna fecha
df_csar.rename(columns = {'Fecha de Entrega':'Fecha de Entrega CSAR'}, inplace = True)

In [10]:
# Concatenar CRC y CRE

# 8.1 Concatenar archivos según Orden de Servicio superior
df_crecrc = pd.concat([df_crc,df_cre], axis = 0)

# 8.2 Eliminar N/A
df_crecrc.dropna(axis = 0, subset = 'Orden de Servicio', inplace = True)

# 8.3 Renombrar columna fecha
df_crecrc.rename(columns = {'Fecha de Entrega':'Fecha de Entrega CRE-CRC'}, inplace = True)

In [11]:
# Cruce Fechas CSAR

# 9.1 Fijar como Key Value = OSO en Planificacion CSAR
df_csar.set_index('Oso', inplace = True)

# 9.2 Fijar como Key Value = OSO en dataframe Base
df1.set_index('OSO', inplace = True)

# 9.3 Join
df1 = df1.join(df_csar, how = 'left')

# 9.4 Reset index
df1.reset_index(inplace = True)

# 9.5 Renombrar Columna
df1.rename(columns = {'index': 'OSO'}, inplace = True)

# 9.6 Ordenar Valores según identificador único
df1.sort_values(by = 'Identificador Unico', inplace = True)

In [12]:
# Cruce Fechas CRC - CRE

# 10.1 Fijar como Key Value = OSO en Planificacion CRC - CRE
df1.set_index('OS.SUP', inplace = True)

# 10.2 Fijar como Key Value = OSO en dataframe Base
df_crecrc.set_index('Orden de Servicio', inplace = True)

# 10.3 Join
df1 = df1.join(df_crecrc, how = 'left')

# 10.4 Reset index y renombrar columna
df1.reset_index(inplace = True)

# 10.5 Renombrar Columna
df1.rename(columns = {'index': 'OS.SUP'}, inplace = True)

# 10.6 Ordenar Valores según identificador único
df1.sort_values(by = 'Identificador Unico', inplace = True)

In [13]:
# Unificar Fechas

# 11.1 Relleno de #NaN con ''
df1['Fecha de Entrega CSAR'].fillna('', inplace = True)
df1['Fecha de Entrega CRE-CRC'].fillna('', inplace = True)

# 11.2 Unir ambas columnas
df1['FECHA DE DESPACHO'] = df1['Fecha de Entrega CSAR'] + df1['Fecha de Entrega CRE-CRC']

# 11.3 Reset Index
df1.reset_index(inplace = True)

# 11.4 Borrar columna index
df1.drop('index', axis = 1, inplace = True)

# 11.5 Eliminar Columa de Fechas CSAR y CRE-CRC
df1.drop(['Fecha de Entrega CSAR', 'Fecha de Entrega CRE-CRC'], axis = 1, inplace = True)

# 11.6 Todo lo que no tenga Plan, FECHA DESPACHO = Vacío
df1.loc[pd.isna(df1['Plan']), 'FECHA DE DESPACHO'] = pd.NA

## Tipo de Cliente

In [14]:
# 12.1 Crear Columna Tipo de Cliente
df1['TIPO DE CLIENTE'] = pd.NA

# 12.2 Crear Columnas auxiliares 4 primeros digitos y 6 primeros digitos
df1['OS.SUP_AUX'] = list(map(lambda x: x[:4], df1['OS.SUP']))
df1['OS.SUP_KIT'] = list(map(lambda x: x[:6], df1['OS.SUP']))

# 12.3 Rellenar Tipo de Cliente
df1.loc[df1['OS.SUP_AUX'] == '5028', 'TIPO DE CLIENTE'] = 'CSAR'
df1.loc[df1['OS.SUP_AUX'] == '5082', 'TIPO DE CLIENTE'] = 'CRE'
df1.loc[df1['OS.SUP_AUX'] == '5059', 'TIPO DE CLIENTE'] = 'CRC'
df1.loc[df1['OS.SUP_KIT'] == '000001', 'TIPO DE CLIENTE'] = 'KIT'

# 12.4 Eliminar Columnas Auxiliares
df1.drop(labels = ['OS.SUP_AUX', 'OS.SUP_KIT'], axis = 1, inplace = True)

## Tipo de OS

In [15]:
# 13.1 Crear Columna Tipo de Cliente
df1['TIPO DE OS'] = pd.NA

## Semana

In [16]:
# 14.1 Crear Columna Semana
df1['Semana'] = pd.NA

# 14.2 Columna auxiliar para arreglar el formato
df1['FECHA DE DESPACHO AUXILIAR'] = df1['FECHA DE DESPACHO']

# 14.3 Fecha Entrega a formato fecha
df1['FECHA DE DESPACHO'] = pd.to_datetime(df1['FECHA DE DESPACHO'], errors = 'coerce', dayfirst = True, format = '%d/%m/%Y')

# 14.4 Calculo de Semana para los valores de la Columna FECHA DE DESPACHO con valores en formato Fecha
df1.loc[pd.isna(df1['FECHA DE DESPACHO']) == False, 'Semana'] = \
list(map(lambda x: datetime.date(x).strftime("%W"), df1.loc[pd.isna(df1['FECHA DE DESPACHO']) == False, 'FECHA DE DESPACHO']))

# 14.5 ELiminar 0 de la derecha a la columna Semana
df1.loc[pd.isna(df1['Semana']) == False,  'Semana'] = \
df1.loc[pd.isna(df1['Semana']) == False,  'Semana'].apply(lambda x: x.lstrip('0'))

# 14.6 FECHA DE DESPACHO a formato Texto
df1['FECHA DE DESPACHO'] = df1['FECHA DE DESPACHO'].dt.strftime("%d/%m/%Y")

# 14.6 Aquellos que no tenian formato fecha , los igualamos a la columna Auxiliar para recuperar dichos valores
df1.loc[pd.isna(df1['FECHA DE DESPACHO']) == True, 'FECHA DE DESPACHO'] = \
df1['FECHA DE DESPACHO AUXILIAR'].iloc[df1.loc[pd.isna(df1['FECHA DE DESPACHO'])].index.values]

# 14.7 Eliminar fecha de despacho Auxiliar
df1.drop('FECHA DE DESPACHO AUXILIAR', axis = 1, inplace = True)

## Activación: Cdt. Activaciones y ULTI. ACTIVACION

In [17]:
# 15 Leer Archivo
df_activacion = pd.read_excel(activacion , dtype = 'string', skipfooter = 5, usecols = ['Orden', 'Fe.sol.'])

In [18]:
# Cantidad de Activaciones

# 16.1 Dataframe para Cantidad de activaciones
ctd_act = df_activacion.loc[:, ['Orden','Fe.sol.']]

# 16.2 Funcion Texto a la Operación
ctd_act['Orden'] = ctd_act['Orden'].apply(lambda x: x.zfill(12))

# 16.3 Crear Orden Superior
ctd_act['OS.SUP'] = [i[:10] + '00' for i in ctd_act['Orden']]

# 16.4 Eliminar columna Orden
ctd_act.drop(columns = 'Orden', inplace = True)

# 16.5 Setear Key Value OS.SUP como Index
ctd_act.set_index('OS.SUP', inplace = True)

# 16.6 Conteo Distinto
ctd_act = ctd_act.groupby('OS.SUP').nunique()

# 16.7 Cambiar nombre de columna
ctd_act.rename(columns = {'Fe.sol.':'Cdt. Activaciones'}, inplace = True)

# 16.8 Setear en df1 Key Value OS.SUP como Index
df1.set_index('OS.SUP', inplace = True)

# 16.9 Join
df1 = df1.join(ctd_act, how = 'left')

# 16.10 Ordenar por Identificador Unico
df1.sort_values(by = 'Identificador Unico', inplace = True)

In [19]:
# Fecha Activación

# 17.1 Dataframe para Cantidad de activaciones
fesol = df_activacion.loc[:, ['Orden','Fe.sol.']]

# 17.2 Funcion Texto a la Operación
fesol['Orden'] = fesol['Orden'].apply(lambda x: x.zfill(12))

# 17.3 Crear Orden Superior
fesol['OS.SUP'] = [i[:10] + '00' for i in fesol['Orden']]

# 17.4 Ordenar por Fecha Solicitud (Mas reciente)
fesol.sort_values(by = 'Fe.sol.', ascending = False, inplace = True)

# 17.5 Eliminar Duplicados
fesol.drop_duplicates('OS.SUP', keep = 'first', inplace = True)

# 17.6 Eliminar 00:00:00 de la Columna Fecha
fesol['Fe.sol.'] = fesol['Fe.sol.'].str.replace(" 00:00:00","", regex = False)

# 17.7 Eliminar Columna Orden
fesol.drop(columns = 'Orden', inplace = True)

# 17.8 Columna fecha a formato fecha
fesol['Fe.sol.'] = pd.to_datetime(fesol['Fe.sol.'])

# 17.9 Columna Fecha a formato String, con formato dd-mm-YYYY
fesol['Fe.sol.'] = fesol['Fe.sol.'].dt.strftime('%d/%m/%Y')

# 17.10 Key Value = OS.Sup en index para realizar el Join
fesol.set_index('OS.SUP', inplace = True)

# 17.11 Renombrar columna
fesol.rename(columns = {'Fe.sol.':'ULTI. ACTIVACION'}, inplace = True)

# 17.12 Join
df1 = df1.join(fesol, how = 'left')

# 17.13 Reset Index
df1.reset_index(inplace = True)

# 17.14 Ordenar por Identificador Unico
df1.sort_values(by = 'Identificador Unico', inplace = True)

## Cruce AFO

In [20]:
# 18. Leer Hojas: Hoja1 y Resumen x Ln
df_afo_resumenxoso = pd.read_excel(afo, sheet_name = 'Hoja1', engine = 'pyxlsb', dtype = 'string')
df_afo_resumenxlinea = pd.read_excel(afo, sheet_name = 'Resumen x Ln', engine = 'pyxlsb', dtype = 'string')

In [21]:
# 19.1 Dataframe con todas aquellas columnas que usan la columna OSO como Key Value
cruce_oso = df_afo_resumenxoso.loc[:, ['OSO', 'St.Consolidacion', 'ST OSO', 'LnPend x OSO']]

# 19.2 Eliminar duplicados de OSO
cruce_oso.drop_duplicates('OSO', keep = 'first', inplace = True)

# 19.3 Fijar OSO como Key value para el Join
cruce_oso.set_index('OSO', inplace = True)

# 19.4 Fijar OSO en dataframe base como Key value para el Join
df1.set_index('OSO', inplace = True)

# 19.5 Join
df1 = df1.join(cruce_oso, how = 'left')

# 19.6 Reset Index
df1.reset_index(inplace = True)

In [22]:
# 20.1 Dataframe con todas aquellas columnas que usan la columna OS+OP+Mat como Key Value
cruce_osopmat = df_afo_resumenxlinea.loc[:, ['OS+OP+Mat', 'ST Linea', 'TSolicitado', 'Delta']]

# 20.2 Eliminar duplicados de OS+OP+Mat
cruce_osopmat.drop_duplicates('OS+OP+Mat', keep = 'first', inplace = True)

# 20.3 Fijar OS+OP+Mat como Key value para el Join
cruce_osopmat.set_index('OS+OP+Mat', inplace = True)

# 20.4 Fijar OS+OP+Mat en dataframe base como Key value para el Join
df1.set_index('OS+OP+MAT', inplace = True)

# 20.5 Join
df1 = df1.join(cruce_osopmat, how = 'left')

# 20.6 Reset Index
df1.reset_index(inplace = True)

# 20.7 Renombrar columna que salió del index
df1.rename(columns = {'index':'OS+OP+MAT'}, inplace = True)

In [23]:
# 21.1 Dataframe con todas aquellas columnas que usan la columna OSO como Key Value
cruce_osup = df_afo_resumenxoso.loc[:, ['Orden Superior', 'ST Padre', 'LnPend x Padre']]

# 21.2 Eliminar duplicados de OS.SUP
cruce_osup.drop_duplicates('Orden Superior', keep = 'first', inplace = True)

# 21.3 Fijar OS.SUP como Key value para el Join
cruce_osup.set_index('Orden Superior', inplace = True)

# 21.4 Fijar OS.SUP en dataframe base como Key value para el Join
df1.set_index('OS.SUP', inplace = True)

# 21.5 Join
df1 = df1.join(cruce_osup, how = 'left')

# 21.6 Reset Index
df1.reset_index(inplace = True)

# 21.7 Renombrar columna que salió del index
df1.rename(columns = {'index':'OS.SUP'}, inplace = True)

In [24]:
# Una vez terminado los cruces:

# 22.1 Ordenar Valores
df1.sort_values(by = 'Identificador Unico', inplace = True)

# 22.2 Renombrar columnas Nuevas
df1.rename(columns = {'St.Consolidacion' : 'CONSOLIDACION OSO', 'ST OSO' : 'ESTATUS OSO', 'LnPend x OSO' : 'LP. X Oso',
            'ST Linea' : 'ESTATUS Linea', 'TSolicitado' : 'SOLICITADO', 'Delta' : 'DELTA', 'ST Padre' : 'ESTATUS Padre', 
            'LnPend x Padre' : 'LP. X Padre'}, inplace = True)

## Conjunto (Todos los clientes) y TIPO OS para CRC

In [25]:
# 23.1 Cargar data día anterior
df_dia_anterior = pd.read_excel(OnHandCompleto_DiaAnterior, sheet_name = 'BASE 1',
                                     usecols = ['OS', 'CONJUNTO','TIPO DE OS'], engine = 'pyxlsb', dtype = 'string')

# 23.2 Renombrar Columna TIPO DE OS. Puesto que haremos el cruce solo para el CRC
df_dia_anterior.rename(columns = {'TIPO DE OS': 'TIPO DE OS CRC'}, inplace = True)

In [26]:
# Conjunto

# 24.1 Dataframe para conjunto
df_conjunto_anterior = df_dia_anterior.loc[:, ['OS', 'CONJUNTO']]

# 24.2 Eliminar valores vacíos
df_conjunto_anterior.dropna(inplace = True)

# 24.3 Eliminar Duplicados
df_conjunto_anterior.drop_duplicates(subset = 'OS', keep = 'first', inplace = True)

# 24.4 Eliminar Conjuntos raros
df_conjunto_anterior.drop(df_conjunto_anterior[df_conjunto_anterior['CONJUNTO'] == '0x2a'].index, inplace = True)

# 24.5 Reordenar Index
df1.sort_index(inplace = True)

# 24.6 Fijar OS como Key value para el Join
df_conjunto_anterior.set_index('OS', inplace = True)

# 24.7 Fijar OS en el dataframe base como Key value para el Join
df1.set_index('OS', inplace = True)

# 24.8 Join
df1 = df1.join(df_conjunto_anterior)

# 24.9 Conjunto = NO MOTOR, para CSAR, CRE y KIT
df1.loc[df1['TIPO DE CLIENTE'] == 'CSAR', 'CONJUNTO'] = 'NO MOTOR'
df1.loc[df1['TIPO DE CLIENTE'] == 'CRE', 'CONJUNTO'] = 'NO MOTOR'
df1.loc[df1['TIPO DE CLIENTE'] == 'KIT', 'CONJUNTO'] = 'NO MOTOR'

In [27]:
# Tipo de OS para CRC

# 25.1 Dataframe para TIPO DE OS CRC
df_os_crc = df_dia_anterior.loc[:, ['OS', 'TIPO DE OS CRC']]

# 25.2 Eliminar valores vacíos
df_os_crc.dropna(inplace = True)

# 25.3 Nos quedamos solo con CRC
df_os_crc = df_os_crc.loc[(df_os_crc['TIPO DE OS CRC'] != 'CSAR') & (df_os_crc['TIPO DE OS CRC'] != 'KIT') \
             & (df_os_crc['TIPO DE OS CRC'] != 'CRE')]

# 25.4 Eliminar duplicados de Orden de Servicio
df_os_crc.drop_duplicates(subset = 'OS', keep = 'first', inplace = True)

# 25.5 Fijar OS como Key value para el Join
df_os_crc.set_index('OS', inplace = True)

# 25.6 Join
df1 = df1.join(df_os_crc)

# 25.7 Tipo de OS para CRC
df1['TIPO DE OS'] = df1['TIPO DE OS CRC']

# 25.8 Tipo de OS CSAR, CRE , KIT
df1.loc[df1['TIPO DE CLIENTE'] == 'CSAR', 'TIPO DE OS'] = 'CSAR'
df1.loc[df1['TIPO DE CLIENTE'] == 'CRE', 'TIPO DE OS'] = 'CRE'
df1.loc[df1['TIPO DE CLIENTE'] == 'KIT', 'TIPO DE OS'] = 'KIT'

# 25.9 Eliminar Columna TIPO DE OS CRC
df1.drop('TIPO DE OS CRC', axis = 1, inplace = True)

# 25.10 Ordenar según identificador Unico
df1.sort_values(by = 'Identificador Unico', inplace = True)

# 25.11 Reset Index
df1.reset_index(inplace = True)

## Job y ESTATUS QPLUS

In [28]:
# 26.1 Leer Archivo
df_qpluss = pd.read_excel(qpluss, usecols = ['OS', 'JobControl', 'Estado', 'OrdenMigación'], dtype = 'string')

In [29]:
# 27.1 Eliminar OS sin valores
df_qpluss.dropna(subset = 'OS', axis = 0, inplace = True)

# 27.2 Dar formato de 12 números a la Orden de Servicio
df_qpluss['OS'] = df_qpluss['OS'].apply(lambda x: x.zfill(12))

# 27.3 Crear columna de Orden Superior con formato correspondiente
df_qpluss['OS.SUP'] = [i[:10] + '00' for i in df_qpluss['OS']]

# 27.4 Reset Index
df_qpluss.reset_index(inplace = True)

# 27.5 Eliminar columna Index
df_qpluss.drop(labels = 'index', axis = 1, inplace = True)

# 27.5 Dar formato a OS.SUP a los valores de Orden Migación
df_qpluss.loc[pd.isna(df_qpluss['OrdenMigación']) == False, 'OS.SUP'] = \
list(map(lambda x: x[:10] + '00' , df_qpluss.loc[pd.isna(df_qpluss['OrdenMigación']) == False, 'OrdenMigación']))

# 27.6 Eliminar columnas OS y OrdenMigación
df_qpluss.drop(['OS', 'OrdenMigación'], axis = 1, inplace = True)

# 27.7 Eliminar duplicados de Orden Superior para hacer el Join
df_qpluss.drop_duplicates(subset = 'OS.SUP', keep = 'first', inplace = True)

# 27.8 Renombrar columnas
df_qpluss.rename(columns = {'JobControl':'JOB', 'Estado':'ESTATUS QPLUS'}, inplace = True)

# 27.9 Setear index a OS.SUP
df_qpluss.set_index('OS.SUP', inplace = True)

# 27.10 Key Values OS.SUP para el dataframe base df1
df1.set_index('OS.SUP', inplace = True)

# 27.11 Join
df1 = df1.join(df_qpluss, how = 'left')

# 27.12 Ordenar Valores según Identificador Unico
df1.sort_values(by = 'Identificador Unico', inplace = True)

# 27.13 Reset Index
df1.reset_index(inplace = True)

# 27.14 Job y Estatus QPLUS = Vacio para los clientes que no nos interesan
df1.loc[pd.isna(df1['TIPO DE CLIENTE']), ['JOB', 'ESTATUS QPLUS']] = pd.NA

## ESTATUS QPLUS CABECERA ECC, ESTATUS QPLUS ECC

In [30]:
# 28 Leer archivo
df_qpluss_ecc = pd.read_excel(qpluss_ecc, dtype = 'string')

In [31]:
# 29.1 Arreglar formato Fecha
df_qpluss_ecc['Fecha entrada'] = df_qpluss_ecc['Fecha entrada'].str.replace(" 00:00:00","", regex = False)
df_qpluss_ecc['Fecha entrada'] = pd.to_datetime(df_qpluss_ecc['Fecha entrada'])

# 29.2 Ordenar fecha por mas Reciente
df_qpluss_ecc.sort_values(by = 'Fecha entrada', ascending = False, inplace = True)

# 29.3 Formato final fecha
df_qpluss_ecc['Fecha entrada'] = df_qpluss_ecc['Fecha entrada'].dt.strftime(date_format = '%d/%m/%Y')

# 29.4 Dar formato de 12 número a la Orden de Servicio
df_qpluss_ecc['Orden'] = df_qpluss_ecc['Orden'].apply(lambda x: x.zfill(12))

# 29.5 Eliminar Duplicados de Orden (Nos quedamos con el primer valor encontrado)
df_qpluss_ecc.drop_duplicates(subset = 'Orden', keep = 'first', inplace = True)

# 29.6 Eliminar Columna Fecha Entrada
df_qpluss_ecc.drop('Fecha entrada', axis = 1, inplace = True)

# 29.7 Renombrar columnas
df_qpluss_ecc.rename(columns = {'Status de usuario' : 'ESTATUS QPLUS ECC', 'Status del sistema':'ESTATUS QPLUS CABECERA ECC'}, 
                     inplace = True)

# 29.8 Columna Orden Como Key Value para el Join con df1
df_qpluss_ecc.set_index('Orden', inplace = True)

# 29.9 Columna OS Como Key Value en df1 para Join con QPlussEcc
df1.set_index('OS', inplace = True)

# 29.10 Join
df1 = df1.join(df_qpluss_ecc, how = 'left')

# 29.11 Reset index
df1.reset_index(inplace = True)

# 29.12 Renombrar columna index
df1.rename(columns = {'index' : 'OS'}, inplace = True)

# 29.13 Ordenar según Identificador Unico
df1.sort_values(by = 'Identificador Unico', inplace = True)

## SIGNIFICADO QPLUS ECC

In [32]:
# 30 Cargar BASE 2
df_base2 = pd.read_excel(OnHandAvance, sheet_name = 'BASE 2', dtype = 'string')
sqecc = df_base2.loc[: , ['ESTATUS QPLUS ECC','SIGNIFICADO QPLUS ECC']]

In [33]:
# DropNA
sqecc.dropna(inplace = True)

In [34]:
# 31.3 Drop Duplicates (Just in Case)
sqecc.drop_duplicates(subset = 'ESTATUS QPLUS ECC', inplace = True)

In [35]:
# 31.4 Key Value 'ESTATUS QPLUS ECC' para el Join con df1
sqecc.set_index('ESTATUS QPLUS ECC', inplace = True)

# 31.5 Key Value 'ESTATUS QPLUS ECC' en df1
df1.set_index('ESTATUS QPLUS ECC', inplace = True)

# 31.6 Join
df1 = df1.join(sqecc, how = 'left')

# 31.7 Reset Index
df1.reset_index(inplace = True)

# 31.8 Ordenar según Identificador Unico
df1.sort_values(by = 'Identificador Unico', inplace = True)

# 31.9 Reset Index
df1.reset_index(inplace = True)

# 31.10 Eliminar columna Index
df1.drop('index', axis = 1, inplace = True)

## Multioso

In [36]:
# 32. Insertar Columna
df1['MULTI-OSO'] = pd.NA

## Rellenar Tipo de OS

In [37]:
# Leer archivo Transaccion ZZSRV_WIP
df_tipo_os = pd.read_excel('TIPO DE OS 15-09.XLSX', dtype = 'string')

# Crear columna Tipo OS para el cruce
df_tipo_os['TIPO DE OS_2'] = pd.NA

# Filtrar los valores que contienen .MO., estos los catalogaremos como Motores
df_tipo_os.loc[df_tipo_os['Denominación'].str.contains('.MO.', regex = False), 'TIPO DE OS_2'] = 'MOTOR'

# El resto, NO MOTOR
df_tipo_os['TIPO DE OS_2'].fillna('NO MOTOR', axis = 0, inplace = True)

In [38]:
# Join
df_tipo_os = df_tipo_os.set_index('Orden')
df1 = df1.set_index('OS')
df1 = df1.join(df_tipo_os, how = 'left')
df1 = df1.reset_index()
df1.rename(columns = {'index': 'OS'}, inplace = True)

In [39]:
# Rellenar Tipo de OS
#df1.loc[(df1['TIPO DE CLIENTE'] == 'CRC') & (pd.isna(df1['TIPO DE OS']) == True) & (pd.isna(df1['TIPO DE OS_2']) == False), 
#        'TIPO DE OS'] = df1.loc[(df1['TIPO DE CLIENTE'] == 'CRC') & (pd.isna(df1['TIPO DE OS']) == True), 'TIPO DE OS_2']

In [40]:
#df1.to_excel('df1.xlsx')

## Arreglos Finales

In [41]:
# 35.1 Ordenar
df1.sort_values(by = 'Identificador Unico', inplace = True)

# 35.2 Reset Index
df1.reset_index(inplace = True)

# 35.3 Eliminar Identificador Unico
df1.drop(labels = 'Identificador Unico', axis = 1, inplace = True)

# 35.4 Orden final de columnas
Columnas_Final = ['OS.SUP','OSO','OS','OP','TIPO DE PROCESO','Tipo almacén','Ubicación','UMp sup.','UMp superior',
                  'Unidad manipulación','Documento','Número de posición','Tipo de documento','Producto',
                  'Descripción de producto','Ctd.','Propietario','Persona autorizada a disponer','Fecha EM','Hora EM',
                  'Tipo de stocks','DIAS EN PROCESO','Plan','FECHA DE DESPACHO','TIPO DE CLIENTE','TIPO DE OS','Semana',
                  'Cdt. Activaciones','ULTI. ACTIVACION','CONSOLIDACION OSO','ESTATUS OSO','ESTATUS Linea','ESTATUS Padre',
                  'SOLICITADO','DELTA','LP. X Oso','LP. X Padre','CONJUNTO','JOB','ESTATUS QPLUS','ESTATUS QPLUS CABECERA ECC',
                  'ESTATUS QPLUS ECC','SIGNIFICADO QPLUS ECC','MULTI-OSO','TU','FCH. Creacion TU','CENTRO. SUMNISTRO']

df1 = df1.reindex(columns = Columnas_Final)

# 35.5 Formato Correcto Columna Cantidad
df1['Ctd.'] = df1['Ctd.'].str.replace(",",".", regex = False)
df1['Ctd.'] = df1['Ctd.'].astype('float')

# 35.6 Formato Correcto Columna Cantidad
df1['Semana'] = df1['Semana'].astype(dtype='Int64')

# 35.7 Formato Correcto Columna Solicitado
df1['SOLICITADO'] = df1['SOLICITADO'].astype(dtype='Int64')

# 35.8 Formato Correcto Columna Delta
df1['DELTA'] = df1['DELTA'].astype(dtype='Int64')

# 35.9 Formato Correcto Columna LP. x Oso
df1['LP. X Oso'] = df1['LP. X Oso'].astype(dtype='Int64')

# 35.10 Formato Correcto Columna LP. X Padre
df1['LP. X Padre'] = df1['LP. X Padre'].astype(dtype='Int64')

# 35.11 CONCAT como primera columna
df1.set_index('OS.SUP', inplace = True)

# 35.12 Espacio entre tablas
df_base2.rename(columns = {'Unnamed: 2':'', 'Unnamed: 6':''}, inplace = True)

# 35.13 ESTATUS QPLUSS ECC como primera columna
df_base2.set_index('ESTATUS QPLUS ECC', inplace = True)

# 35.14 BASE1 y BASE2 con index
df1 = df1.reset_index()
df_base2 = df_base2.reset_index()

In [42]:
#df1.to_excel('df1.xlsx')

In [43]:
#df1.to_excel('2023-04-03 ON HAND COMPLETO.xlsx')

In [44]:
# 36 Descargar Archivo
today = date.today()
fecha_hoy = today.strftime("%Y-%m-%d")
Nombre_Archivo = fecha_hoy + ' ON HAND COMPLETO.xlsx'
with pd.ExcelWriter(Nombre_Archivo) as writer:  
    df1.to_excel(writer, sheet_name='BASE 1', index = False)
    df_base2.to_excel(writer, sheet_name='BASE 2', index = False)

In [45]:
# 37 Abrir excel y modificarlo

# Abrir Instancia Excel
xlApp = win32.Dispatch('Excel.Application')
xlApp.Visible = True

# Abrir On Hand Completo (Libro)
path = os.path.abspath(Nombre_Archivo)
wb = xlApp.Workbooks.Open(path)

# Ajustar Columnas
wb.Worksheets(1).Activate()
wb.ActiveSheet.Columns.AutoFit()

wb.Worksheets(2).Activate()
wb.ActiveSheet.Columns.AutoFit()

# Guardar como XLSB
wb.SaveAs(Filename = os.path.dirname(path) + '\\' + fecha_hoy + ' ON HAND COMPLETO', FileFormat = 50)